# Arbre de décision à partir de rien

## Introduction

Les arbres de décision sont une méthode de **prédiction** puissante et extrêmement populaire.

Ils sont populaires parce que le **modèle final** est si facile à comprendre par les praticiens et les experts du domaine de l'apprentissage supervisé. L'arbre de décision final peut expliquer exactement pourquoi une **prédiction** spécifique a été faite, ce qui le rend très attrayant pour une utilisation opérationnelle.

Les arbres de décision fournissent également la base pour des méthodes d'ensemble plus avancées telles que les **forêts aléatoires**(Random Forest).

### Le but de ce mini-projet est de mettre en application les notions vues en cours :

1. Comment calculer et évaluer les points de partage pour une donnée ?
2. Comment organiser des divisions en une structure d'arbre de décision ?
3. Comment appliquer l'algorithme de classification à un problème réel ?

## Description

Un **noeud** représente une seule variable d'entrée (X) et un point de partage sur cette variable, en supposant que la variable est numérique. Les **nœuds feuilles** (également appelés nœuds terminaux) de l'arbre contiennent une variable de sortie (y) qui est utilisée pour effectuer une **prédiction**.

Une fois créé, un arbre peut être parcouru avec une nouvelle rangée de données suivant chaque branche avec les divisions jusqu'à ce qu'une **prédiction** finale soit faite.

La répartition avec le meilleur coût (coût le plus bas car nous minimisons les coûts) est sélectionnée. Ici, la fonction de minimisation sera l'**entropie**, c'est le **principe de minimisation**. Toutes les variables d'entrée et tous les points de partage possibles sont évalués et choisis de manière gourmande en fonction de la fonction de coût.


## Jeu de données

L'ensemble de données correspond à des données météorologiques consiste à prédire si les données correpondent à une **classe** N comme négative ou P pour positive. Un exemple serait une utilisation agricole pour différents types de plantation que l'on peut posséder.

L'ensemble de données contient 14 lignes avec 4 variables qualitatives. C'est un problème de classification avec deux classes (classification binaire).

Vous trouverez ci-dessous une liste des quatre variables de l'ensemble de données.

* **Outlook** (variable qualitative : sunny, overcast, rain).
* **Temprature** (variable qualitative : hot, cold, mid).
* **Humidity** (variable qualitative : high, normal, low).
* **Windy** (variable qualitative : false, true).


Voici le contenu du jeu de données :

In [46]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd

df = pd.read_csv("test_data.csv")
df







,outlook,temperature,humidity,windy,C
0,sunny,hot,high,False,N
1,sunny,hot,high,True,N
2,overcast,hot,high,False,P
3,rain,mid,high,False,P
4,rain,cold,normal,True,P
5,rain,cold,normal,True,P
6,overcast,cold,high,False,N
7,sunny,mid,normal,False,P
8,sunny,cold,normal,False,P
9,rain,mid,normal,False,P


## Indice d'entropie

L'**index d'entropie** est le nom de la fonction de coût utilisée pour évaluer les divisions dans l'ensemble de données dans notre cas (d'autres fonctions peuvent être utilisé comme la fonction Gini).

Un **score de l'entropie** donne une idée de la **qualité** d'une scission en fonction du mélange des classes dans les deux groupes créés par la scission.

Nous devons d'abord calculer la proportion de classes dans chaque groupe.

*proportion = count(class_value) / count(rows)*